In [14]:
import pandas as pd
import os
import itertools
from itertools import product
import numpy as np
import re 

os.getcwd()

'C:\\Users\\nschw\\OneDrive\\QSS 20\\final project'

In [15]:
##Drop all irrelevant columns in whd data
def column_clean(df):
    to_keep = ['case_violtn_cnt', 'naics_code_description', 'findings_end_date',
           'findings_start_date', 'case_id', 'h2a_violtn_cnt', 'h2a_bw_atp_amt',
           'h2a_ee_atp_cnt','h2a_cmp_assd_amt', 'st_cd', "ld_dt", "naics_cd"]   
    to_remove = [column for column in df.columns if column not in to_keep]
    return(df.drop(columns = to_remove))

In [16]:
whd = pd.read_csv("WHD data.csv")

C:\Users\nschw\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
# start of whd cleaning
whd = column_clean(whd)
##Subset to non-zero h2a violations 
whd = whd[~whd.h2a_violtn_cnt.astype(str).str.contains('0')]

In [18]:
whd = whd[~whd.ld_dt.astype(str).str.contains('1\d{3}', regex = True)]
whd = whd[~whd.ld_dt.astype(str).str.contains('200[0-9]', regex = True)]
whd = whd[~whd.ld_dt.astype(str).str.contains('201[0-1]', regex = True)]
whd = whd[~whd.ld_dt.astype(str).str.contains('2021', regex = True)]

In [19]:
# Create year and quarter columns in WHD data
whd["start_year"] = [int(date_string[-4:]) for date_string in whd.findings_start_date]
whd["end_year"] = [int(date_string[-4:]) for date_string in whd.findings_end_date]

regex = r"\d{1,2}"
whd["start_month_int"] = [int(re.match(regex, date_string).group()) for date_string in whd.findings_start_date]
whd["end_month_int"] = [int(re.match(regex, date_string).group()) for date_string in whd.findings_end_date]

month_to_qtr_dict = {1:"MAR", 2:"MAR", 3:"MAR", 4:"JUN", 5:"JUN", 6:"JUN", 7:"SEP", 8:"SEP", 9:"SEP", 10:"DEC", 11:"DEC", 12:"DEC"}
whd["start_quarter"] = [month_to_qtr_dict[month] for month in whd.start_month_int]
whd["end_quarter"] = [month_to_qtr_dict[month] for month in whd.end_month_int]


whd["ld_dt_object"] = pd.to_datetime(whd["ld_dt"])
whd["ld_dt_year"] = whd["ld_dt_object"].dt.year.astype(int)
whd["ld_dt_month"] = whd["ld_dt_object"].dt.month.astype(int)
whd["ld_dt_quarter"] = [month_to_qtr_dict[month] for month in whd.ld_dt_month]

C:\Users\nschw\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\nschw\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [20]:
whd.to_csv("whd_for_merge")

In [21]:
whd

,case_id,st_cd,naics_code_description,case_violtn_cnt,findings_start_date,findings_end_date,h2a_violtn_cnt,h2a_bw_atp_amt,h2a_ee_atp_cnt,h2a_cmp_assd_amt,...,start_year,end_year,start_month_int,end_month_int,start_quarter,end_quarter,ld_dt_object,ld_dt_year,ld_dt_month,ld_dt_quarter
72,1802853,FL,Sugarcane Farming,3,10/1/2015,10/2/2016,2,0.00,0,0.0,...,2015,2016,10,10,DEC,DEC,2017-03-14 01:00:03,2017,3,MAR
158,1770830,NY,Corn Farming,9,5/21/2014,11/1/2015,9,4011.10,8,900.0,...,2014,2015,5,11,JUN,DEC,2016-06-11 02:22:25,2016,6,JUN
174,1762079,MS,Berry (except Strawberry) Farming,7,5/1/2014,6/11/2015,7,0.00,0,3150.0,...,2014,2015,5,6,JUN,JUN,2016-02-19 01:00:04,2016,2,MAR
910,1518046,SC,Other Noncitrus Fruit Farming,36,6/19/2008,6/19/2008,36,0.00,0,0.0,...,2008,2008,6,6,JUN,JUN,2015-02-20 01:00:06,2015,2,MAR
1434,1689982,GA,Farm Labor Contractors and Crew Leaders,6,11/5/2012,4/25/2013,6,0.00,0,1800.0,...,2012,2013,11,4,DEC,JUN,2016-02-19 01:00:04,2016,2,MAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310528,1749730,FL,Orange Groves,104,1/1/2014,1/17/2015,65,10050.80,35,45300.0,...,2014,2015,1,1,MAR,MAR,2020-10-07 13:04:27,2020,10,DEC
310557,1824992,FL,Farm Labor Contractors and Crew Leaders,169,4/28/2017,7/6/2017,136,115574.90,64,29512.4,...,2017,2017,4,7,JUN,SEP,2020-10-07 13:04:28,2020,10,DEC
310559,1825302,GA,Farm Labor Contractors and Crew Leaders,7,2/1/2017,12/31/2017,7,0.00,0,26395.2,...,2017,2017,2,12,MAR,DEC,2020-10-07 13:04:28,2020,10,DEC
310582,1830745,MI,Berry (except Strawberry) Farming,393,7/1/2016,9/15/2018,364,67633.76,333,2876.4,...,2016,2018,7,9,SEP,SEP,2020-10-07 13:04:28,2020,10,DEC
